<a href="https://colab.research.google.com/github/PranavShashidhara/Movie-Recommendation-system/blob/main/Movie_Movie_Collaberative_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install sortedcontainers

In [5]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from datetime import datetime
from sortedcontainers import SortedList
import pickle
import os
with open("user2movie.json", "rb") as f:
  user2movie = pickle.load(f)
with open("movie2user.json", "rb") as f:
  movie2user = pickle.load(f)
with open("usermovie2rating.json","rb") as f:
  usermovie2rating = pickle.load(f)
with open("usermovie2rating_test.json", "rb") as f:
  usermovie2rating_test = pickle.load(f)

N = np.max(list(user2movie.keys())) + 1
m1 = np.max(list(movie2user.keys()))
m2 = np.max([m for (u, m), r in usermovie2rating_test.items()])
M = max(m1, m2)
if M > 2000:
  print("N = ", N, "are you sure you want to continue? This will take a very long time to run depending on the size of the data.")
  print("If so please comment out this code.")
  exit()

In [25]:
K = 20 # number of neighbors we'd like to consider
limit = 5 # no.of common movies to check to be considered
neighbors = [] # To store negihbors in the list
averages = [] # each item's average rating for later use.
deviation = [] # each item's deviation for later use

for i in range(M):
  user_i = movie2user[i]
  user_i_set = set(user_i)
  ratings_i ={user:float(usermovie2rating[(user, i)]) for user in user_i}
  avg_i = np.mean(list(ratings_i.values()))
  dev_i = {movie:(float(ratings_i) - avg_i) for movie, ratings_i in ratings_i.items()}
  dev_i_values = np.array(list(dev_i.values()))
  sigma_i = np.sqrt(dev_i_values.dot(dev_i_values)) # To get the magnitude value.
  averages.append(avg_i)
  deviation.append(dev_i)

  sl = SortedList()
  for j in range(M): # Looping for all the movies without including anything.
    if j != i:
      user_j = movie2user[j]
      user_j_set = set(user_j)
      common_users = (user_i_set & user_j_set) # getting common users for those movies
      if len(common_users) > limit:
        ratings_j = {user:float(usermovie2rating[(user, j)]) for user in user_j}
        avg_j = np.mean(list(ratings_j.values()))
        dev_j = {movie:(float(ratings_j) - avg_j) for movie, ratings_j in ratings_j.items()}
        dev_j_values = np.array(list(dev_j.values()))
        sigma_j = np.sqrt(dev_j_values.dot(dev_j_values))

        numerator = sum(dev_i[m]*dev_j[m] for m in user_i_set & user_j_set)
        w_ij = numerator /(sigma_i * sigma_j)

        # insert into sorted list and truncate
        # list is sorted in ascending order so max value
        sl.add((-w_ij, j))
        if len(sl) > K:
          del sl[-1]
  neighbors.append(sl)
  if i % 1 ==0:
    print(i)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198


In [38]:
def predict(i, u):
  numerator = 0
  denominator = 0
  if i < len(neighbors):

    for neg_w, j in neighbors[i]:
      try:
        numerator += -neg_w * deviation[j][u]
        denominator += abs(neg_w)
      except KeyError:
        pass
  if denominator == 0:
    if i < len(averages):  # Check if average is available for the movie
          prediction = averages[i]
    else:
          prediction = np.mean(averages) if averages else 3
  else:
    prediction = numerator/denominator + averages[i]
  prediction = min(5, prediction)
  prediction = max(0.5, prediction)
  return prediction

train_predictions = []
train_targets = []
for (u, m), targets in usermovie2rating.items():
  predictions = predict(m, u)
  train_predictions.append(predictions)
  train_targets.append(targets)

test_predictions = []
test_targets = []
for (u, m), targets in usermovie2rating_test.items():
  predictions = predict(m, u)
  test_predictions.append(predictions)
  test_targets.append(targets)

def mse(p,t):
  p = np.array(p)
  t = np.array(t, dtype=np.float64)
  return np.mean((p-t)**2)

print('train mse:', mse(train_predictions, train_targets))
print('test mse:', mse(test_predictions, test_targets))

train mse: 0.7332156679018316
test mse: 0.8131916014546953
